In [1]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import cv2

tf.get_logger().setLevel('ERROR')
print(f'Using TensorFlow v{tf.__version__}')

Using TensorFlow v2.4.0


In [2]:
with open('./assets/instances_val2017.json') as f:
    dic = json.load(f)
cats = dic['categories']

In [3]:
path = './assets/IMG_8389.JPG'
img = cv2.imread(path)
img = cv2.resize(img, (0, 0), fx=0.2, fy=0.2)
img_tensor = np.expand_dims(img, axis=0)






Loading model...
Model loaded
Analyzing image...
Image analyzed


In [ ]:
print('Loading model...')
detector = hub.load("https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512_kpts/1")
print('Model loaded')

print('Analyzing image...')
result = detector(img_tensor)
print('Image analyzed')

In [4]:
def bound_label(width, height, det_box, det_idx):
    x1, y1 = int(det_box[0][1] * width), int(det_box[0][0] * height)
    x2, y2 = int(det_box[0][3] * width), int(det_box[0][2] * height)

    _ = cv2.rectangle(im_draw,
                      (x1, y1),
                      (x2, y2),
                      (0, 255, 0),
                      thickness=2)
    _ = cv2.putText(im_draw,
                    get_lbl(det_idx),
                    (int((x2 + x1) / 2) - 30, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 255, 0), 2)
    box_pts_so_far.append([x1, y1])


def get_lbl(curr):
    res_idx = result['detection_classes'].numpy()[0][curr]
    res = filter(lambda x: x['id'] == res_idx, cats)
    return list(res)[0]['name']

In [ ]:
book_count = 0
box_pts_so_far = []

# image to display
im_draw = img.copy()
width = im_draw.shape[1]
height = im_draw.shape[0]

# only classify confident images
res_scores = result['detection_scores'].numpy()[0]
# threshold (only take classifications with this or greater confidence)
thresh = 0.18
res_scores_confident = np.extract(res_scores >= thresh, res_scores)
print('Original scores: ', res_scores)
print('Confident scores: ', res_scores_confident)

boxes = result['detection_boxes'].numpy()

# tolerance (pixels) between top left detections
# higher tolerance is less books counted
tolerance = 0


# check if a bounding box is within a certain x, y tolerance to be counted as the same book
def good_box(bx):
    good = True
    x1, y1 = int(bx[0][1] * width), int(bx[0][0] * height)
    for pt in box_pts_so_far:
        good = abs(x1 - pt[0]) >= tolerance and abs(y1 - pt[1]) >= tolerance
        if not good:
            break
    return good


for det_idx in range(len(res_scores_confident)):
    box = boxes[:, det_idx, :]
    if get_lbl(det_idx) == "book" and good_box(box):
        bound_label(width, height, box, det_idx)
        book_count += 1

cv2.putText(im_draw, f'Books: {book_count}', (im_draw.shape[1] - 140, im_draw.shape[0] - 30),
            cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 2)

cv2.imshow("Analyzed", im_draw)
cv2.waitKey(0)

Original scores:  [0.37023804 0.34426802 0.3134058  0.31316328 0.299519   0.29925472
 0.29026008 0.27294168 0.24799562 0.22251144 0.208561   0.19929907
 0.19740593 0.18985835 0.17593944 0.17417106 0.16785374 0.15720814
 0.14259258 0.12554961 0.11501276 0.11385912 0.11011595 0.09650937
 0.09589851 0.09365949 0.09299746 0.08893877 0.08609852 0.08587015
 0.08160517 0.07831895 0.07521957 0.06936061 0.06673166 0.05962634
 0.05842853 0.05778667 0.05616915 0.05377981 0.0530901  0.05281153
 0.05070516 0.05009025 0.05005103 0.04879585 0.04855418 0.04779768
 0.04712978 0.04631257 0.04627615 0.04333517 0.0425365  0.0420858
 0.04194438 0.04178807 0.04117534 0.04048437 0.04046732 0.04027984
 0.03983277 0.03942087 0.0387488  0.03851607 0.03834599 0.03833455
 0.03806099 0.03801602 0.03779337 0.03639311 0.03621039 0.03586844
 0.0352436  0.03520334 0.03462392 0.03440154 0.03397003 0.03386879
 0.03353217 0.03345549 0.03339043 0.03320858 0.03316593 0.03311944
 0.03293824 0.03218165 0.03216046 0.03166789 